# 1. Initialization

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import warnings

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             fbeta_score, make_scorer, classification_report, confusion_matrix)
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler

plt.style.use('fivethirtyeight')
%matplotlib inline

In [45]:
seed = 42

def create_train_test_sets(data, test_size=.2, downsample=False, downsample_fracs=None, binary_class=None):
    if downsample_fracs is not None:
        for c, f in downsample_fracs.items():
            remove_mask = data[data['crashSeverity'] == c].sample(frac=f, random_state=seed).index
            data = data.drop(remove_mask)
            
    y = data['crashSeverity']
    X = data.drop('crashSeverity', axis=1)
    
    if binary_class is not None:
        y = downsampled_ohe['crashSeverity'].apply(lambda _: 1 if _ == binary_class else 0)

    if downsample:
        rus = RandomUnderSampler(random_state=seed)
        X, y = rus.fit_sample(X, y)

    return train_test_split(X, y, test_size=test_size, random_state=seed)

In [13]:
crash_data_clean = pd.read_csv('Crash_Analysis_System_CAS_data_clean.csv', keep_default_na=False)

In [14]:
data = crash_data_clean.drop(['cornerRoadSideRoad', 'crashRPSH', 'directionRoleDescription'], axis=1)

In [15]:
def parse_type(dtype):
    if dtype == 'int':
        return np.int8
    elif dtype == 'float':
        return np.float
    else:
        return dtype

# Read features descriptions
features_catalog = pd.read_table('features_description.tsv')
# Make a dict to use as dtypes for panda's dataframe
features_dtypes = features_catalog.set_index('feature_name')['pandas_dtype'].apply(parse_type).to_dict()
# Keep only the columns that remain in the clean version of the dataframe
features_dtypes = {k: v for k, v in features_dtypes.items() if k in data.columns}

In [16]:
data = data.astype(features_dtypes, copy=False)

In [17]:
data['speedLimit'] = data['speedLimit'].apply(lambda x: 999 if x == -1 else x)

In [18]:
categorical_features = [f for f, t in features_dtypes.items() if t  == 'category']
categorical_features.remove('crashSeverity')
data_ohe = pd.get_dummies(data,columns=categorical_features)

# 2. Preprocessing

Talk about no need for feature scaling or transformation, nor normalizaton
Why one hot encoding and not label encoding, why not ordinal labels and the need to have categorical features binarized

Explore and choose the right metric

<span style="color:red">THIS SHOULD GO TO 1</span>

The features `fatalCount`, `seriousInjuryCount`, `minorInjuryCount` are provided by the NZTA after the crash has been processed. Their values are not available at the time of the crah. At the same time, their values are almost completely correlated to our target variable, naturally.

For example, a *fatal* crash will have a `fatalCount > 0`. But a *minor* crash will have `fatalCount` and `seriousInjuryCount` with a value of `0`. This can be seen for example, by training a Naive Bayes classifier while including the above features. Notice how both, the training and test performance metrics are high. This is due to data lakeage coming from this particular features.

In [19]:
# THIS SHOULD GO TO 1
data_ohe.drop(['fatalCount', 'seriousInjuryCount', 'minorInjuryCount'], axis=1, inplace=True)

Before we build we train our models, we need to address the imbalance of classes present in our dataset:

In [21]:
data_ohe['crashSeverity'].value_counts()

N    444953
M    140864
S     34914
F      5855
Name: crashSeverity, dtype: int64

If we expect to get a usable model, we need to treat this situation somehow. To that end, and considering the vast amount crashes of class `N`, we will downsample this specific class. A downsample of 70% will remove a lot of the class'samples and yet it will still remain the most frequent one:

In [50]:
down_fracs={'N': .9, 'M': .5}
X_train, X_test, y_train, y_test = create_train_test_sets(data_ohe, test_size=.2, downsample_fracs=down_fracs)

In [51]:
pd.Series(y_train).value_counts()

M    56485
N    35455
S    27961
F     4655
Name: crashSeverity, dtype: int64

# 3. Benchmark

Before we continue, a word on the performance metrics we will be using to evaluate the different models below.

Let us consider what metrics we should use. Towards that goal, we need to think about the applications of the predictions we want to make. Namely, given a fresh call informing an emergency agency of a car crash that just took place, we want to predict its severity so that the agency is able to dispatch the appropriate response. Here we are assuming that at this point in time we are provided with all the features present in the dataset. However, we understand that this sceneario is not always the case and so, a fully productionized impementation should incorporate a better handling of possible missing values (which we have cleaned and filled in for this project).

The most simple metric that comes to mind naturally is *Accuracy*. That is, the ratio of the correctly predicted crashes among all samples. This includes both *True Positives* and *True Negatives*. Although, since we are in a multiclass case, we will perform a *One vs All classification* for each class –and then average the results. For clarification, the formula for accuracy is as follows:

$$Accuracy=\sum_{i=1}^n x^i$$

<span style="color: red">Fix this and add other metrics</span>

<span style="color: red">Explain whi we will use anf f1 score. Arugemnt for precision and recall both being important.

In [28]:
def print_results(true, pred, betta=1, digits=2, average=None):
    print('Accuracy score: ', format(accuracy_score(true, pred)))
    print('Precision score: ', format(precision_score(true, pred, average=average)))
    print('Recall score: ', format(recall_score(true, pred, average=average)))
    print('F1 score: ', format(f1_score(true, pred, average=average)))
    print('F betta score with betta=%.2f: ' % betta, format(fbeta_score(true, pred, betta, average=average)))
    print('\n', classification_report(y_test, pred, digits=digits))

First, we train a Naive Bayes classifier to use as a benchmark. As such, we won't be doing any hyperparameter tunning just yet. Its performance will be used as a baseline that we will work to improve.

In [53]:
clf_NB_benchmark = MultinomialNB()
clf_NB_benchmark.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
clf_NB_benchmark = joblib.load('clf_NB_benchmark.pkl') 

In [ ]:
joblib.dump(clf_NB_benchmark, 'clf_NB_benchmark.pkl')

Next, lets throw in our test set and print some performance metrics:

In [54]:
predictions_NB = clf_NB_benchmark.predict(X_test)
print_results(y_test, predictions_NB, average='weighted')

Accuracy score:  0.40786769428387926
Precision score:  0.45591294020598605
Recall score:  0.40786769428387926
F1 score:  0.41342028316644336
F betta score with betta=1.00:  0.41342028316644336

              precision    recall  f1-score   support

          F       0.10      0.50      0.17      1200
          M       0.52      0.37      0.43     13947
          N       0.48      0.62      0.54      9040
          S       0.36      0.20      0.26      6953

avg / total       0.46      0.41      0.41     31140



we see that all our reference metrics hover around the same values of 0.65.

However, if we consider the same metrics at each class level, we see quite a different picture. First, we can clearly see that most of the weight in the global metric values comes from the `N` class. This is logical, considering most of the sample belong to this class. For each of the other three classes, we see values much lower.

Lets remember that if a crash is of class `N`, then dispatching a response apprpriate for a fatal crash will not produce much inconvenience other than an economical one. However, if the crash is of class `F`, then dispatching a response team equiped to treat only a `N` crash is a serious problem.

Looking for example at the metrics for class `F` we see a precision of only 3% and a recall of 49%. From the definition of precision and recall, this means that the `FP` for this class are much higher than the `TP` –thus producing a precision of 0.03— and that there are almost as much `TP` as there are `FN` –producing a recall value of 0.49.

So, from the fact that there are almost as many `TP` as there are `FN` we conclude that we are missing a lot of relevant values –in fact, almost hanlf of them. On the other hand, the fact there are a lot more `FP` than `TP` tells us that the model is missclisifying a lot of crashes as being of class `F`.

The case for classes `M` and `S` are similar.

So, for any model that we train, we need to check that our performance metrics have an even behaviour across all classes. That is, ideally all the metrics should present low variability across classes, but at least the F1 score should present such behavior.

# 4. Classifiers

In this section we will train a few model with different algorithms and search for the one that performs best. In order to choose the best performing algorithm, we will carefully analyze the same metrics from the previous section, plus AUC/ROC to compare between the models.

Also, for each algorithm we will first train a model with all default values to get a baseline. Then we will do a grid search with relevant hyperparameter values to find the optimal combination.

<span style="color:red">explain how trees and ensembles are good wwhen we don't know the importance and relevance of features</span>

## 4.1 Random Forest

<span style="color: red">explain pros and cons of RF for this problem</spam>

In [55]:
clf_RF_baseline = RandomForestClassifier(random_state=seed, n_jobs=-1)
clf_RF_baseline.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [ ]:
clf_RF_baseline = joblib.load('clf_RF_baseline.pkl') 

In [ ]:
joblib.dump(clf_RF_baseline, 'clf_RF_baseline.pkl') 

In [56]:
predictions_RF = clf_RF_baseline.predict(X_test)
print_results(y_test, predictions_RF, digits=5, average='weighted')

Accuracy score:  0.4785806037251124
Precision score:  0.4653496750627261
Recall score:  0.4785806037251124
F1 score:  0.4660370952966163
F betta score with betta=1.00:  0.4660370952966163

              precision    recall  f1-score   support

          F    0.21544   0.10000   0.13660      1200
          M    0.49838   0.61891   0.55215     13947
          N    0.51894   0.47445   0.49569      9040
          S    0.37255   0.26780   0.31161      6953

avg / total    0.46535   0.47858   0.46604     31140



Similar to the benchmark classifier from section 3, we see that class `N` it greatly responsible for the final values of all metrics. But in this case, the behavior for the other classes is more disparate. Both precision and recall are way to low for classes `F`, `S` and `M`.

So lets run a grid search. In this case we will try to find optimal values for the following hypermarameters:
* Number of estimator
* Max depth for trees
* Minimum samples in leaf required to evaluate further splitting.

For the number of estimators we will use values `[10, 20, 30, 40, 50]`. We should remember that each estimator is a decision tree built with a *bag* of samples ramdomly selected.

Max depth is the parameter that will allow the trees to become more precise on their clasification, as each level further distills the impiruty of the samples than end up on each leaf. However, this parameters is also very much related to the overfitting of the trees. Although we are growing a forest in part to address this partial overfitting, we should aim for a balance. So, Considering the vast amount of features available, we will use values `[7, 9, 11, 13, 15, 17, 19]`.

Finally, the minimum number on sample in a leaf required to perform another split is also a parameters related to overfitting and how pure are the samples in the final leaf nodes of the trees. Considering the size of the training set –about 43K samples– we will use the values `[50, 100, 500, 1000]` for the search.

This combination of parameters will produce different model. We will use the default cross validation with a 3-fold to have good enough cross validation results without incurring in to much processing time; and F1 score to evaluate performance.

Next, we run the search, time it and report on the results:

In [57]:
import time
warnings.filterwarnings('ignore')

parameters = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [50, 80, 100, 200],
    'min_samples_split': [2, 10, 20, 50]
}

start_time = time.time()

clf_RF_gridsearch = RandomForestClassifier(random_state=42, n_jobs=-1)
scorer = make_scorer(fbeta_score, beta=1, average='weighted')
grid_obj_RF = GridSearchCV(clf_RF_gridsearch, parameters, scorer, verbose=4)
grid_obj_RF = grid_obj_RF.fit(X_train, y_train)

end_time = time.time()

Fitting 3 folds for each of 64 candidates, totalling 192 fits
[CV] max_depth=50, min_samples_split=2, n_estimators=50 ..............
[CV]  max_depth=50, min_samples_split=2, n_estimators=50, score=0.4770026475155838, total=   6.1s
[CV] max_depth=50, min_samples_split=2, n_estimators=50 ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.2s remaining:    0.0s


[CV]  max_depth=50, min_samples_split=2, n_estimators=50, score=0.4760372987231359, total=   6.7s
[CV] max_depth=50, min_samples_split=2, n_estimators=50 ..............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.1s remaining:    0.0s


[CV]  max_depth=50, min_samples_split=2, n_estimators=50, score=0.47770911366026275, total=   6.0s
[CV] max_depth=50, min_samples_split=2, n_estimators=100 .............


...

[CV]  max_depth=50, min_samples_split=2, n_estimators=100, score=0.47970031945357905, total=  11.0s
[CV] max_depth=50, min_samples_split=2, n_estimators=100 .............
[CV]  max_depth=50, min_samples_split=2, n_estimators=100, score=0.47798013834254965, total=  11.3s
[CV] max_depth=50, min_samples_split=2, n_estimators=100 .............
[CV]  max_depth=50, min_samples_split=2, n_estimators=100, score=0.47981050939543035, total=  11.1s
[CV] max_depth=50, min_samples_split=2, n_estimators=200 .............
[CV]  max_depth=50, min_samples_split=2, n_estimators=200, score=0.4812403189371037, total=  22.2s
[CV] max_depth=50, min_samples_split=2, n_estimators=200 .............
[CV]  max_depth=50, min_samples_split=2, n_estimators=200, score=0.47837414051293004, total=  21.6s
[CV] max_depth=50, min_samples_split=2, n_estimators=200 .............
[CV]  max_depth=50, min_samples_split=2, n_estimators=200, score=0.48247015158345524, total=  21.7s
[CV] max_depth=50, min_samples_split=2, n_esti

[CV]  max_depth=80, min_samples_split=2, n_estimators=100, score=0.4779606839790718, total=  13.2s
[CV] max_depth=80, min_samples_split=2, n_estimators=100 .............
[CV]  max_depth=80, min_samples_split=2, n_estimators=100, score=0.4770738218973275, total=  12.6s
[CV] max_depth=80, min_samples_split=2, n_estimators=100 .............
[CV]  max_depth=80, min_samples_split=2, n_estimators=100, score=0.47878194372055527, total=  14.1s
[CV] max_depth=80, min_samples_split=2, n_estimators=200 .............
[CV]  max_depth=80, min_samples_split=2, n_estimators=200, score=0.47806161078774684, total=  24.8s
[CV] max_depth=80, min_samples_split=2, n_estimators=200 .............
[CV]  max_depth=80, min_samples_split=2, n_estimators=200, score=0.47854885841459066, total=  24.4s
[CV] max_depth=80, min_samples_split=2, n_estimators=200 .............
[CV]  max_depth=80, min_samples_split=2, n_estimators=200, score=0.480787512294394, total=  22.0s
[CV] max_depth=80, min_samples_split=2, n_estimat

[CV]  max_depth=100, min_samples_split=2, n_estimators=100, score=0.47881591576889027, total=  11.2s
[CV] max_depth=100, min_samples_split=2, n_estimators=100 ............
[CV]  max_depth=100, min_samples_split=2, n_estimators=100, score=0.47753591194849315, total=  11.4s
[CV] max_depth=100, min_samples_split=2, n_estimators=100 ............
[CV]  max_depth=100, min_samples_split=2, n_estimators=100, score=0.480695888026194, total=  11.3s
[CV] max_depth=100, min_samples_split=2, n_estimators=200 ............
[CV]  max_depth=100, min_samples_split=2, n_estimators=200, score=0.48031406764719814, total=  22.3s
[CV] max_depth=100, min_samples_split=2, n_estimators=200 ............
[CV]  max_depth=100, min_samples_split=2, n_estimators=200, score=0.47957259367301835, total=  21.9s
[CV] max_depth=100, min_samples_split=2, n_estimators=200 ............
[CV]  max_depth=100, min_samples_split=2, n_estimators=200, score=0.4800588095738166, total=  22.0s
[CV] max_depth=100, min_samples_split=2, n

[CV]  max_depth=200, min_samples_split=2, n_estimators=100, score=0.47881591576889027, total=  11.4s
[CV] max_depth=200, min_samples_split=2, n_estimators=100 ............
[CV]  max_depth=200, min_samples_split=2, n_estimators=100, score=0.47753591194849315, total=  11.4s
[CV] max_depth=200, min_samples_split=2, n_estimators=100 ............
[CV]  max_depth=200, min_samples_split=2, n_estimators=100, score=0.48071572714708644, total=  11.8s
[CV] max_depth=200, min_samples_split=2, n_estimators=200 ............
[CV]  max_depth=200, min_samples_split=2, n_estimators=200, score=0.4803153679323024, total=  23.7s
[CV] max_depth=200, min_samples_split=2, n_estimators=200 ............
[CV]  max_depth=200, min_samples_split=2, n_estimators=200, score=0.47957259367301835, total=  24.4s
[CV] max_depth=200, min_samples_split=2, n_estimators=200 ............
[CV]  max_depth=200, min_samples_split=2, n_estimators=200, score=0.4800588095738166, total=  25.0s
[CV] max_depth=200, min_samples_split=2, 

[Parallel(n_jobs=1)]: Done 192 out of 192 | elapsed: 59.8min finished


In [59]:
grid_obj_RF.best_params_

{'max_depth': 50, 'min_samples_split': 20, 'n_estimators': 200}

In [61]:
best_rf = grid_obj_RF.best_estimator_
best_rf.set_params(class_weight=None)
best_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [62]:
predictions_best_RF = best_rf.predict(X_test)
print_results(y_test, predictions_best_RF, digits=5, average='weighted')

Accuracy score:  0.5244380218368657
Precision score:  0.5211566492292301
Recall score:  0.5244380218368657
F1 score:  0.4955682574381463
F betta score with betta=1.00:  0.4955682574381463

              precision    recall  f1-score   support

          F    0.44444   0.03667   0.06774      1200
          M    0.50828   0.73944   0.60245     13947
          N    0.59873   0.49204   0.54017      9040
          S    0.45936   0.21947   0.29703      6953

avg / total    0.52116   0.52444   0.49557     31140

